In [11]:
import logging
from sys import stdout

formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler(stdout)
console_handler.setFormatter(formatter)

logger = logging.getLogger('opensky.spark_consumer')
logger.addHandler(console_handler)
logger.setLevel('INFO')


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pandas as pd
from pyspark.sql import types as T
from pyspark.sql import functions as F
import json
import os


# # Topics/Brokers
topic_real_time_states = "real-time-states"
topic_sparse_states = 'sparse_states'
topic_enriched_real_time_states = 'enriched-real-rime-states'
broker = "localhost:9092"

host_name = 'cnt7-naya-cdh6'
hive_host = "localhost"
hdfs_host = "localhost"
hdfs_port = 8020

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.1 pyspark-shell'

spark = SparkSession \
    .builder \
    .appName("StructuredRealTimeState") \
    .getOrCreate()


schema = T.ArrayType(T.StructType()\
                     .add("time", T.TimestampType())\
                     .add("icao24", T.StringType())\
                     .add("callsign", T.StringType())\
                     .add("last_contact", T.TimestampType())\
                     .add("longitude", T.FloatType())\
                     .add("latitude", T.FloatType())\
                     .add("baro_altitude", T.FloatType())\
                     .add("on_ground", T.IntegerType())\
                     .add("velocity", T.FloatType())\
                     .add("geo_altitude", T.FloatType())\
                     .add("squawk", T.StringType())\
                     .add("position_source", T.IntegerType()))

In [13]:
import pyarrow as pa
import re
from datetime import datetime, timedelta
import urllib
from impala.dbapi import connect

def drop_old_partitions(impala_conn: connect, table_name:str, table_src_path:str, 
                             partition_name:str, earliest_time_to_keep:datetime.timestamp):
    """
    Drop old partitions at path <table_src_path>, drop all partitions older than <earliest_time_to_keep>
    First drop partitions through impala client (impala.dbapi), then delete folders
    """
    logger.debug(f'dropping old partitions for table {table_name}, all partitions oldert than {earliest_time_to_keep}')
    
    fs = pa.hdfs.connect(
            host=hdfs_host, 
            port=hdfs_port, 
            user='hdfs', 
            kerb_ticket=None, 
            driver='libhdfs', 
            extra_conf=None)


    partition_pattern = r'(.+\=(.+))'
    partitions_paths = fs.ls(table_src_path)
    partitions_tup = [re.findall(partition_pattern, partition_path) for partition_path in partitions_paths]
    # partitions_tup is of form : (partition path, partition date)
    # urllib.parse.unquote in order to turn special chars as %3A to regular (:) , aslo flatten the list
    partitions_tup = [(pt[0][0], urllib.parse.unquote(pt[0][1])) for pt in partitions_tup if len(pt) > 0]
    partitions_dict = {date : path for path, date in partitions_tup}

    partitions_to_delete = [p_d for p_d in partitions_dict.keys() if datetime.strptime(p_d, '%Y-%m-%d %H:%M:%S') < earliest_time_to_keep]

    # here partitions to_delete holds all partitions that should be deleted

    crsr = impala_conn.cursor()

    try:
        for part_key in partitions_to_delete:
            crsr.execute(f'alter table {table_name} drop if exists partition ({partition_name}="{part_key}");')
            fs.delete(partitions_dict[part_key], recursive=True)
            logger.debug(f'deleted : {partitions_dict[part_key]}')
    except Exception as ex:
        logger.Error(ex)
    finally:
        crsr.close()


import pyarrow as pa
from datetime import datetime, timedelta
from impala.dbapi import connect
from os import path


def write_to_tables(df: DataFrame, epoch_id):
    last_hour_table_name = 'last_hour'
    target_database = 'opensky_network'
    
    root_data_path = '/user/naya/FinalProject/'
    last_hour_path = path.join(root_data_path, 'last_hour')
    last_day_path = path.join(root_data_path, 'last_day')
    last_week_path = path.join(root_data_path,'last_week')
    
    logger.info(f'write_to_hive: epoch_id: {epoch_id} len: {df.count()}')
    # write to each table - minutes, hours, days
    if df.count() != 0:
        df.persist()
        logger.debug(f'trying to write to : {last_hour_path}')
        df.withColumn('date_minute', F.date_trunc('minute', df.time))\
                    .write\
                    .mode("append")\
                    .partitionBy('date_minute')\
                    .parquet(f'hdfs://localhost:8020/{last_hour_path}')
        logger.debug(f'Trying to write to : {last_day_path}')
        df.withColumn('date_hour', F.date_trunc('hour', df.time))\
                    .write\
                    .mode("append")\
                    .partitionBy('date_hour')\
                    .parquet(f'hdfs://localhost:8020/{last_day_path}')
        df.unpersist()

        impala_conn = connect(host=host_name, database = target_database, user = 'naya', password = 'naya', auth_mechanism = 'NOSASL')

        drop_old_partitions(impala_conn, 'states_last_hour', last_hour_path, 
                                 'date_minute', datetime.now() - timedelta(hours=1))
        drop_old_partitions(impala_conn, 'states_last_day', last_day_path, 
                                 'date_hour', datetime.now() - timedelta(hours=24))

        impala_crsr = impala_conn.cursor()
        try:
            for table_name in ['states_last_hour', 'states_last_day']:
                impala_crsr.execute(f'alter table {table_name} recover partitions;')
                impala_crsr.execute(f'refresh {table_name};')
        except Exception as ex:
            logger.error(ex)
        finally:
            impala_crsr.close()
            
#         crsr.execute(f'show partitions {table_name};')
#         logger.debug([d for d, *rest in crsr.fetchall()])
    
#     impala_client.table(last_hour_table_name).drop_partition('date_minute=2019-12-28 12:13:00')
#     impala_client.table(last_hour_table_name).alter()
#     impala_client.table(last_hour_table_name).refresh()



In [14]:
import geopandas as gpd
import matplotlib
import numpy as np
import pandas as pd
from multiprocessing import  Pool
from shapely.geometry.point import Point as ShapelyPoint
from pyspark.sql.functions import udf
import time

# Add country to each row
# Load countries file
COUNTRIES_GDF = gpd.read_file('./Material/ne_10m_admin_0_countries.shp')

def get_country(lon:float, lat:float) -> str:
    if lon is None or lat is None:
        return ""
    
    #convert to aheply point
    point = ShapelyPoint(lon, lat)
    country = COUNTRIES_GDF['NAME'][COUNTRIES_GDF['geometry'].contains(point)]
    # Need to handle cases when no country found
    country = country.item() if len(country) > 0 else ""

    return country

def to_json_cols(cols_list):
    out_dict = {}
    for col in cols_list:
        F.to_json(col)

udf_get_country = udf(get_country, T.StringType())


## TBD ##
def enrich_add_country_column(df, epoch_id):
    logger.debug(f'in enrich_add_country_column epoch_id:{epoch_id}, with len(df): {df.count()}')
    if df is None or df.count() <= 0:
        return
    minidf = df.limit(100)
    start_enrich = time.time()
#     df = df.withColumn('country', udf_get_country('longitude', 'latitude'))
    df = minidf.withColumn('country', udf_get_country('longitude', 'latitude'))
    end_enrich = time.time()
    logger.debug(f'enriched {df.count()} records, within {end_enrich - start_enrich} seconds')
    test_df = df.toJSON().toDF('value')
    logger.debug(f'this is the df: {test_df.show()}')
#     rdd = df.rdd.map(lambda row: row.asDict())
#     logger.debug(f'rdd is {rdd.collect()}')
    # send to kafka
#     df.select(F.toJSON())\
#         .write\
#         .format("kafka") \
#         .option("kafka.bootstrap.servers", broker) \
#         .option("topic", topic_enriched_real_time_states) \
#         .save()


In [15]:
from pyspark.sql import functions as F

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", broker) \
    .option("subscribe", topic_real_time_states) \
    .option("startingOffsets", "latest")\
    .load()

state_vectors_df = df.select(F.from_json(F.col("value").cast("string"), schema).alias("value"))\
                        .select((F.explode("value").alias("value")))\
                        .select("value.*")

# # Write to parquet file
# TBD - handle target path creation
parquet_path = 'hdfs://cnt7-naya-cdh6.org:8020/FinalProject/parquet_archive'
parquet_checkpoint_path = "/home/naya/parquet_checkpoint"
parquet_write = state_vectors_df\
                .withColumn('date_hour', F.date_trunc('hour', state_vectors_df.time))\
                .writeStream\
                .outputMode("append")\
                .format("parquet")\
                .partitionBy('date_hour')\
                .option("checkpointLocation", parquet_checkpoint_path)\
                .option("path", parquet_path)\
                .start()

tables_write = state_vectors_df\
                .writeStream\
                .foreachBatch(write_to_tables)\
                .start()

# # TBD # #
# enrich_with_countries = state_vectors_df\
#             .writeStream\
#             .foreachBatch(enrich_add_country_column)\
#             .start()


2020-01-16 01:28:36,001 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 0 len: 0
2020-01-16 01:28:36,001 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 0 len: 0
2020-01-16 01:28:55,406 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 1 len: 12793
2020-01-16 01:28:55,406 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 1 len: 12793
2020-01-16 01:29:09,455 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2 len: 12792
2020-01-16 01:29:09,455 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2 len: 12792
2020-01-16 01:29:25,802 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 3 len: 12833
2020-01-16 01:29:25,802 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 3 len: 12833
2020-01-16 01:29:42,211 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4 len: 12808
2020-01-16 01:29:42,211 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 4 len: 12808
2020-01-16 01:29:57,043 - opensky.spark_consumer

2020-01-16 01:47:39,572 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 42 len: 12875
2020-01-16 01:47:57,648 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 43 len: 12871
2020-01-16 01:47:57,648 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 43 len: 12871
2020-01-16 01:48:16,140 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 44 len: 12864
2020-01-16 01:48:16,140 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 44 len: 12864
2020-01-16 01:48:34,095 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 45 len: 12862
2020-01-16 01:48:34,095 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 45 len: 12862
2020-01-16 01:48:54,304 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 46 len: 12886
2020-01-16 01:48:54,304 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 46 len: 12886
2020-01-16 01:49:23,475 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 47 len: 12915
2020-01-16 01:49:23,475 - open

2020-01-16 02:04:41,845 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 85 len: 11934
2020-01-16 02:04:41,845 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 85 len: 11934
2020-01-16 02:05:09,146 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 86 len: 11902
2020-01-16 02:05:09,146 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 86 len: 11902
2020-01-16 02:05:37,578 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 87 len: 11826
2020-01-16 02:05:37,578 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 87 len: 11826
2020-01-16 02:06:03,498 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 88 len: 11848
2020-01-16 02:06:03,498 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 88 len: 11848
2020-01-16 02:06:25,511 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 89 len: 11822
2020-01-16 02:06:25,511 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 89 len: 11822
2020-01-16 02:06:50,544 - open

2020-01-16 02:20:21,301 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 127 len: 13863
2020-01-16 02:20:21,301 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 127 len: 13863
2020-01-16 02:20:45,234 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 128 len: 14899
2020-01-16 02:20:45,234 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 128 len: 14899
2020-01-16 02:21:20,748 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 129 len: 15234
2020-01-16 02:21:20,748 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 129 len: 15234
2020-01-16 02:26:34,179 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 130 len: 15359
2020-01-16 02:26:34,179 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 130 len: 15359
2020-01-16 02:27:01,960 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 131 len: 15019
2020-01-16 02:27:01,960 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 131 len: 15019
2020-01-16 02:27:25,

2020-01-16 02:39:43,680 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 169 len: 6140
2020-01-16 02:40:06,134 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 170 len: 6119
2020-01-16 02:40:06,134 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 170 len: 6119
2020-01-16 02:40:28,510 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 171 len: 6142
2020-01-16 02:40:28,510 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 171 len: 6142
2020-01-16 02:40:51,121 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 172 len: 6131
2020-01-16 02:40:51,121 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 172 len: 6131
2020-01-16 02:41:14,254 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 173 len: 6140
2020-01-16 02:41:14,254 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 173 len: 6140
2020-01-16 02:41:26,138 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 174 len: 6154
2020-01-16 02:41:26,138 - open

...

2020-01-16 13:10:22,115 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2363 len: 4831
2020-01-16 13:10:34,792 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2364 len: 4835
2020-01-16 13:10:34,792 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2364 len: 4835
2020-01-16 13:10:47,303 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2365 len: 4842
2020-01-16 13:10:47,303 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2365 len: 4842
2020-01-16 13:11:00,222 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2366 len: 4839
2020-01-16 13:11:00,222 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2366 len: 4839
2020-01-16 13:11:13,175 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2367 len: 4848
2020-01-16 13:11:13,175 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2367 len: 4848
2020-01-16 13:11:27,409 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2368 len: 4853
2020-01-16 13:11:27,

2020-01-16 13:21:15,157 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2405 len: 5108
2020-01-16 13:21:33,599 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2406 len: 5102
2020-01-16 13:21:33,599 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2406 len: 5102
2020-01-16 13:21:51,731 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2407 len: 5082
2020-01-16 13:21:51,731 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2407 len: 5082
2020-01-16 13:22:11,234 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2408 len: 5071
2020-01-16 13:22:11,234 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2408 len: 5071
2020-01-16 13:22:30,151 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2409 len: 5079
2020-01-16 13:22:30,151 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2409 len: 5079
2020-01-16 13:22:49,237 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2410 len: 5098
2020-01-16 13:22:49,

2020-01-16 13:34:39,753 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2447 len: 5200
2020-01-16 13:34:52,931 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2448 len: 5198
2020-01-16 13:34:52,931 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2448 len: 5198
2020-01-16 13:35:06,748 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2449 len: 5188
2020-01-16 13:35:06,748 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2449 len: 5188
2020-01-16 13:35:20,785 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2450 len: 5185
2020-01-16 13:35:20,785 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2450 len: 5185
2020-01-16 13:35:35,279 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2451 len: 5198
2020-01-16 13:35:35,279 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2451 len: 5198
2020-01-16 13:35:49,766 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2452 len: 5196
2020-01-16 13:35:49,

2020-01-16 13:46:15,004 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2489 len: 5320
2020-01-16 13:46:34,692 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2490 len: 5326
2020-01-16 13:46:34,692 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2490 len: 5326
2020-01-16 13:46:54,002 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2491 len: 5332
2020-01-16 13:46:54,002 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2491 len: 5332
2020-01-16 13:47:13,222 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2492 len: 5328
2020-01-16 13:47:13,222 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2492 len: 5328
2020-01-16 13:47:32,764 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2493 len: 5334
2020-01-16 13:47:32,764 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2493 len: 5334
2020-01-16 13:47:52,829 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2494 len: 5329
2020-01-16 13:47:52,

2020-01-16 13:59:05,786 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2531 len: 5454
2020-01-16 13:59:20,236 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2532 len: 5450
2020-01-16 13:59:20,236 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2532 len: 5450
2020-01-16 13:59:35,787 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2533 len: 5451
2020-01-16 13:59:35,787 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2533 len: 5451
2020-01-16 13:59:50,944 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2534 len: 5470
2020-01-16 13:59:50,944 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2534 len: 5470
2020-01-16 14:00:05,487 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2535 len: 5465
2020-01-16 14:00:05,487 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2535 len: 5465
2020-01-16 14:00:20,533 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2536 len: 5470
2020-01-16 14:00:20,

In [18]:
print(f'parquet_write stream status is active: {parquet_write.isActive}')
print(f'tables_write stream status is active: {tables_write.isActive}')

parquet_write stream status is active: True
tables_write stream status is active: True
2020-01-16 14:08:44,088 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2564 len: 5575
2020-01-16 14:08:44,088 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2564 len: 5575
2020-01-16 14:09:03,292 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2565 len: 5570
2020-01-16 14:09:03,292 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2565 len: 5570
2020-01-16 14:09:23,787 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2566 len: 5597
2020-01-16 14:09:23,787 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2566 len: 5597
2020-01-16 14:09:43,338 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2567 len: 5598
2020-01-16 14:09:43,338 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2567 len: 5598
2020-01-16 14:10:03,800 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2568 len: 5611
2020-01-16 14:10:03,800 - opens

2020-01-16 14:21:11,122 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2605 len: 5807
2020-01-16 14:21:25,474 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2606 len: 5819
2020-01-16 14:21:25,474 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2606 len: 5819
2020-01-16 14:21:40,405 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2607 len: 5829
2020-01-16 14:21:40,405 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2607 len: 5829
2020-01-16 14:21:55,543 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2608 len: 5823
2020-01-16 14:21:55,543 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2608 len: 5823
2020-01-16 14:22:11,155 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2609 len: 5799
2020-01-16 14:22:11,155 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2609 len: 5799
2020-01-16 14:22:28,950 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2610 len: 5800
2020-01-16 14:22:28,

2020-01-16 14:34:05,147 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2647 len: 5984
2020-01-16 14:34:28,566 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2648 len: 5991
2020-01-16 14:34:28,566 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2648 len: 5991
2020-01-16 14:34:50,181 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2649 len: 6011
2020-01-16 14:34:50,181 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2649 len: 6011
2020-01-16 14:35:11,955 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2650 len: 6020
2020-01-16 14:35:11,955 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2650 len: 6020
2020-01-16 14:35:33,943 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2651 len: 5988
2020-01-16 14:35:33,943 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2651 len: 5988
2020-01-16 14:35:56,556 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2652 len: 5996
2020-01-16 14:35:56,

2020-01-16 14:45:58,714 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2689 len: 5998
2020-01-16 14:46:16,117 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2690 len: 6024
2020-01-16 14:46:16,117 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2690 len: 6024
2020-01-16 14:46:34,312 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2691 len: 6036
2020-01-16 14:46:34,312 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2691 len: 6036
2020-01-16 14:46:52,559 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2692 len: 6049
2020-01-16 14:46:52,559 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2692 len: 6049
2020-01-16 14:47:10,752 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2693 len: 6059
2020-01-16 14:47:10,752 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2693 len: 6059
2020-01-16 14:47:28,874 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2694 len: 6066
2020-01-16 14:47:28,

2020-01-16 14:59:39,551 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2731 len: 6154
2020-01-16 14:59:53,498 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2732 len: 6158
2020-01-16 14:59:53,498 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2732 len: 6158
2020-01-16 15:00:08,018 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2733 len: 6162
2020-01-16 15:00:08,018 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2733 len: 6162
2020-01-16 15:00:22,320 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2734 len: 6157
2020-01-16 15:00:22,320 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2734 len: 6157
2020-01-16 15:00:37,067 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2735 len: 6168
2020-01-16 15:00:37,067 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2735 len: 6168
2020-01-16 15:00:52,073 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2736 len: 6175
2020-01-16 15:00:52,

2020-01-16 15:11:52,975 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2773 len: 6482
2020-01-16 15:12:13,544 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2774 len: 6504
2020-01-16 15:12:13,544 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2774 len: 6504
2020-01-16 15:12:34,861 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2775 len: 6527
2020-01-16 15:12:34,861 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2775 len: 6527
2020-01-16 15:12:55,909 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2776 len: 6524
2020-01-16 15:12:55,909 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2776 len: 6524
2020-01-16 15:13:17,315 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2777 len: 6501
2020-01-16 15:13:17,315 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2777 len: 6501
2020-01-16 15:13:39,320 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2778 len: 6517
2020-01-16 15:13:39,

2020-01-16 15:24:24,410 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2815 len: 6602
2020-01-16 15:24:43,066 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2816 len: 6600
2020-01-16 15:24:43,066 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2816 len: 6600
2020-01-16 15:25:03,871 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2817 len: 6586
2020-01-16 15:25:03,871 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2817 len: 6586
2020-01-16 15:25:22,346 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2818 len: 6578
2020-01-16 15:25:22,346 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2818 len: 6578
2020-01-16 15:25:40,609 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2819 len: 6567
2020-01-16 15:25:40,609 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2819 len: 6567
2020-01-16 15:25:59,218 - opensky.spark_consumer - INFO - write_to_hive: epoch_id: 2820 len: 6558
2020-01-16 15:25:59,

In [19]:
# stop streams
parquet_write.stop()
tables_write.stop()
print(f'parquet_write stream status is active: {parquet_write.isActive}')
print(f'tables_write stream status is active: {tables_write.isActive}')

parquet_write stream status is active: False
tables_write stream status is active: False
